In [91]:
import matplotlib.pyplot as plt
import numpy as np
import qiskit as qs
from qiskit.circuit.library import MCMTGate, ZGate, phase_estimation

In [88]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator
from qiskit_ibm_runtime.fake_provider import FakeMelbourneV2
from qiskit.primitives import StatevectorSampler

hardware = False

if hardware:
    fake_melbourne = FakeMelbourneV2()
    pm = generate_preset_pass_manager(backend=fake_melbourne, optimization_level=1)
    options = {"simulator": {"seed_simulator": 42}}
    sampler = Sampler(mode=fake_melbourne, options=options)
else:
    sampler = StatevectorSampler()

In [64]:
def grover_operator(oracle, n_input_qubits):
    n = oracle.num_qubits
    
    grover_diff = qs.QuantumCircuit(n)
    
    for i in range(n_input_qubits):
        grover_diff.h(i)
        grover_diff.x(i)
    grover_diff.append(MCMTGate(ZGate(), n_input_qubits-1, 1), list(range(n_input_qubits)))
    for i in range(n_input_qubits):
        grover_diff.x(i)
        grover_diff.h(i)
    
    return oracle.compose(grover_diff, list(range(n)))

def initial_state(n_input_qubits, n_aux_qubits):
    n = n_input_qubits+n_aux_qubits+1
    init = qs.QuantumCircuit(n)
    for i in range(n_input_qubits):
        init.h(i)
    init.x(n-1)
    init.h(n-1)
    return init

def opt_r(N, k):
    r_float = np.pi/4/np.arcsin(np.sqrt(k/N)) - 1/2
    return int(np.round(r_float))

In [74]:
n_input_qubits = 4
n_aux_qubits = 4
n = n_input_qubits+n_aux_qubits+1
N = 2**n_input_qubits

oracle = qs.QuantumCircuit(n)

oracle.x(0)
oracle.x(2)
oracle.append(MCMTGate(ZGate(), n-1, 1), list(range(n)))
oracle.x(2)
oracle.x(0)

oracle.draw()

┌───┐ 1-Z ┌───┐
q_0: ┤ X ├──■──┤ X ├
     └───┘  │  └───┘
q_1: ───────■───────
     ┌───┐  │  ┌───┐
q_2: ┤ X ├──■──┤ X ├
     └───┘  │  └───┘
q_3: ───────■───────

In [103]:
def binary(k, bits):
    if k >= 2**bits:
        raise Exception("k cannot be larger than 2^bits-1")
    _binary = [int(x) for x in bin(k)[2:]]
    return np.append(np.zeros(bits-len(_binary), dtype=int), np.array(_binary))

def check_site(site_ind, n_enc_qubits):
    site_bin = binary(site_ind, n_enc_qubits)

    qc = qs.QuantumCircuit(n_enc_qubits+3)

    for i in range(n_enc_qubits):
        if site_bin[i] == 0:
            qc.x(i)
    qc.mcx(list(range(n_enc_qubits)), n_enc_qubits)

    qc.cx(n_enc_qubits+2, n_enc_qubits+1)
    qc.x(n_enc_qubits+1)
    qc.mcx([n_enc_qubits, n_enc_qubits+1], n_enc_qubits+1)
    
    qc.mcx(list(range(n_enc_qubits)), n_enc_qubits)
    for i in range(n_enc_qubits):
        if site_bin[i] == 0:
            qc.x(i)

    return qc

check_site(4)

[1, 0, 0]

In [87]:
qc = initial_state(n)

r = opt_r(N, 1)

gr_op = grover_operator(oracle)

for i in range(r):
    qc = qc.compose(gr_op, list(range(n)))

qc.measure_all()

In [89]:
shots = 2048
if hardware:
    job = sampler.run([pm.run(qc)], shots=shots)
else:
    job = sampler.run([qc], shots=shots)
result = job.result()[0].data.meas.get_counts()

In [90]:
result

{'1010': 1025,
 '0110': 63,
 '0011': 68,
 '0100': 63,
 '1000': 61,
 '1111': 67,
 '1100': 60,
 '1101': 47,
 '1011': 63,
 '1001': 87,
 '0000': 78,
 '0010': 67,
 '0111': 85,
 '0101': 69,
 '1110': 71,
 '0001': 74}

In [98]:
gr_op = grover_operator(oracle)

n_est = 1

qc = qs.QuantumCircuit(n+n_est)

for i in range(n):
    qc.h(i)

qc.compose(phase_estimation(n_est, gr_op), list(range(n+n_est)))


qc.measure_all()

In [99]:
shots = 2048
if hardware:
    job = sampler.run([pm.run(qc)], shots=shots)
else:
    job = sampler.run([qc], shots=shots)
result = job.result()[0].data.meas.get_counts()

In [100]:
result

{'01000': 130,
 '00010': 123,
 '00110': 137,
 '01101': 125,
 '00011': 127,
 '01001': 129,
 '01011': 136,
 '00111': 103,
 '01110': 136,
 '00000': 121,
 '01100': 140,
 '00001': 126,
 '00101': 135,
 '01111': 134,
 '00100': 126,
 '01010': 120}